**Library Import**

In [6]:
import os
import cv2
from detectron2.engine import DefaultTrainer, HookBase
from detectron2.evaluation import COCOEvaluator
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.data.datasets import register_coco_instances
import matplotlib.pyplot as plt

**prepare the dataset**

In [7]:
import json
import os
from sklearn.model_selection import train_test_split

# Load the original COCO annotations JSON file
with open("/home/gs285/AIPI_HW/Faster RCNN/annotations_COCO/laptop_annotation.json") as f:
    coco_data = json.load(f)

# Get all image IDs from the dataset
image_ids = [img['id'] for img in coco_data['images']]

# Split into 80% training and 20% validation
train_ids, val_ids = train_test_split(image_ids, test_size=0.2, random_state=42)

def filter_annotations_by_image_ids(coco_data, image_ids):
    # Filter images
    filtered_images = [img for img in coco_data['images'] if img['id'] in image_ids]
    
    # Filter annotations
    filtered_annotations = [ann for ann in coco_data['annotations'] if ann['image_id'] in image_ids]
    
    # Preserve the other fields (categories, etc.)
    filtered_data = {
        "images": filtered_images,
        "annotations": filtered_annotations,
        "categories": coco_data['categories']
    }
    return filtered_data

# Create new JSON files for train and val sets
train_data = filter_annotations_by_image_ids(coco_data, train_ids)
val_data = filter_annotations_by_image_ids(coco_data, val_ids)

# Save the new annotation files
os.makedirs("split_annotation", exist_ok=True)
with open("split_annotation/annotations_train.json", "w") as f:
    json.dump(train_data, f)

with open("split_annotation/annotations_val.json", "w") as f:
    json.dump(val_data, f)


**Visualization during the training phase on validation set**

In [8]:
# Hook to evaluate and visualize the validation set every 100 iterations
class ValidationVisualizationHook(HookBase):
    def __init__(self, cfg, predictor, eval_period=500):
        self.cfg = cfg
        self.predictor = predictor
        self.eval_period = eval_period

    def after_step(self):
        iteration = self.trainer.iter
        if iteration % self.eval_period == 0 and iteration != 0:
            self.visualize_predictions()

    def visualize_predictions(self):
        # Get validation dataset
        dataset_dicts = DatasetCatalog.get(self.cfg.DATASETS.TEST[0])
        metadata = MetadataCatalog.get(self.cfg.DATASETS.TEST[0])

        # Visualize predictions on the first 5 validation images
        for d in dataset_dicts[:5]:
            img = cv2.imread(d["file_name"])
            outputs = self.predictor(img)
            
            # Print predictions to check if the model is producing results
            print("Predictions:", outputs)

            # Visualize the predicted bounding boxes
            visualizer = Visualizer(img[:, :, ::-1], metadata=metadata, scale=0.8, instance_mode=ColorMode.IMAGE)
            out = visualizer.draw_instance_predictions(outputs["instances"].to("cpu"))

            # Convert the visualized output to an image suitable for matplotlib
            result_image = out.get_image()[:, :, ::-1]

            # Display the image using matplotlib
            plt.figure(figsize=(12, 8))
            plt.imshow(result_image)
            plt.axis("off")  # Turn off axis
            plt.show()  # Display the image in the notebook


**Detector Training**

In [ ]:
class Detector:
    def __init__(self):
        # Register datasets
        register_coco_instances("my_dataset_train", {}, "/home/gs285/AIPI_HW/Faster RCNN/split_annotation/annotations_train.json", "/home/gs285/AIPI_HW/image_dataset")
        register_coco_instances("my_dataset_val", {}, "/home/gs285/AIPI_HW/Faster RCNN/split_annotation/annotations_val.json", "/home/gs285/AIPI_HW/image_dataset")

        # Create configuration
        self.cfg = get_cfg()
        self.cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
        self.cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")
        self.cfg.DATASETS.TRAIN = ("my_dataset_train",)
        self.cfg.DATASETS.TEST = ("my_dataset_val",)
        self.cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
        self.cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3
        self.cfg.MODEL.DEVICE = 'cuda'
        self.cfg.SOLVER.MAX_ITER = 3000
        self.cfg.SOLVER.IMS_PER_BATCH = 2
        self.cfg.SOLVER.BASE_LR = 0.00025
        self.cfg.SOLVER.WARMUP_ITERS = 0
        self.cfg.SOLVER.CLIP_GRADIENTS.ENABLED = True
        self.cfg.SOLVER.CLIP_GRADIENTS.CLIP_VALUE = 1.0
        self.cfg.OUTPUT_DIR = "./output"

        # Create trainer and predictor
        self.trainer = CustomTrainer(self.cfg)
        self.predictor = DefaultPredictor(self.cfg)

        # Add custom hooks (visualize predictions every 100 iterations)
        self.trainer.register_hooks([ValidationVisualizationHook(self.cfg, self.predictor, eval_period=500)])

    def train(self):
        # Resume or start training
        self.trainer.resume_or_load(resume=False)
        self.trainer.train()

    def evaluate(self):
        # Run evaluation on validation set
        self.trainer.test(self.cfg, self.trainer.model)

class CustomTrainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name):
        return COCOEvaluator(dataset_name, cfg, False, output_dir=cfg.OUTPUT_DIR)

# Instantiate and run the detector
detector = Detector()
detector.train()
detector.evaluate()


[10/14 19:10:55 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (4, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (4,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (12, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (12,) in the model! You might want to double check if this is expected.
Some model parameters or buffers are not found in the checkpoint:
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, 

[10/14 19:10:56 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from https://dl.fbaipublicfiles.com/detectron2/COCO-Detection/faster_rcnn_R_50_FPN_3x/137849458/model_final_280758.pkl ...
